In [ ]:
# Fixed imports (add missing Dataset import)
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torch
import torchvision

# --- Fixed transform section (remove duplicate definition) ---
def repeat_channel(x):
    return x.repeat(3, 1, 1)  # (C, H, W) → (3, H, W)

# Single transform definition with all steps
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Lambda(repeat_channel),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- Fixed Dataset class ---
class CTScanDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        # Add sorted() for consistent ordering
        self.image_paths = sorted([
            os.path.join(root_dir, f) 
            for f in os.listdir(root_dir) 
            if f.endswith('.png')
        ])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx])
        if self.transform:
            image = self.transform(image)
        return image

# --- Added error handling for data loading ---
try:
    dataset = CTScanDataset(root_dir='hemorrhage_slices', transform=transform)
    dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
except FileNotFoundError as e:
    raise SystemExit(f"Error loading data: {str(e)}")

# --- Improved model loading ---
# --- Fixed model loading section ---
try:
    device = torch.device('cpu')
    
    # 1. Define model architecture (MUST match training architecture)
    class CTScanModel(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.base_model = torchvision.models.resnet18(pretrained=False)
            self.base_model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.classifier = torch.nn.Linear(512, 2)
        def forward(self, x):
            x = self.base_model(x)
            x = self.classifier(x)
            return x

    
    # 2. Initialize model
    model = CTScanModel()
    
    # 3. Load state dict
    state_dict = torch.load('pytorch_model.bin', map_location=device)
    model.load_state_dict(state_dict)
    
    # 4. Set to eval mode
    model.eval()
    model.to(device)

except Exception as e:
    raise ValueError(f"Model loading failed: {str(e)}") from e


# --- Prediction with tensor device management ---
predictions = []
with torch.inference_mode():
    for batch in dataloader:
        batch = batch.to(device)
        outputs = model(batch)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

# Final predictions
class_labels = ['normal', 'abnormal']  # Update with your actual classes
final_predictions = [class_labels[pred] for pred in predictions]

C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_17560\649720087.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('pytorch_model.bin', map_location

ValueError: Model loading failed: Error(s) in loading state_dict for CTScanModel:
	Missing key(s) in state_dict: "base_model.conv1.weight", "base_model.bn1.weight", "base_model.bn1.bias", "base_model.bn1.running_mean", "base_model.bn1.running_var", "base_model.layer1.0.conv1.weight", "base_model.layer1.0.bn1.weight", "base_model.layer1.0.bn1.bias", "base_model.layer1.0.bn1.running_mean", "base_model.layer1.0.bn1.running_var", "base_model.layer1.0.conv2.weight", "base_model.layer1.0.bn2.weight", "base_model.layer1.0.bn2.bias", "base_model.layer1.0.bn2.running_mean", "base_model.layer1.0.bn2.running_var", "base_model.layer1.1.conv1.weight", "base_model.layer1.1.bn1.weight", "base_model.layer1.1.bn1.bias", "base_model.layer1.1.bn1.running_mean", "base_model.layer1.1.bn1.running_var", "base_model.layer1.1.conv2.weight", "base_model.layer1.1.bn2.weight", "base_model.layer1.1.bn2.bias", "base_model.layer1.1.bn2.running_mean", "base_model.layer1.1.bn2.running_var", "base_model.layer2.0.conv1.